In [28]:
import time
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
import datetime

Data Loading

In [29]:
ratings_train = pd.read_csv("./Data/rate_train.csv", low_memory=False)
ratings_test = pd.read_csv("./Data/rate_test.csv",low_memory=False)

In [30]:
datasets_train = ratings_train.Node_Id.unique()
model_train = ratings_train.Model_Id.unique()
datasets_test = ratings_test.Node_Id.unique()
model_test = ratings_test.Model_Id.unique()
meta_models = pd.read_csv("./Data/model_v.csv",low_memory=False)
models = meta_models.model_id.unique()

Rating Matrix

In [31]:
data_model_train_matrix = pd.DataFrame(index=datasets_train,columns=models)
data_model_test_matrix = pd.DataFrame(index=datasets_test,columns=models)

In [32]:
for row in ratings_train.itertuples():
    data_model_train_matrix.loc[row[1]][row[2]] = row[3]
data_model_train_matrix = data_model_train_matrix.fillna(0)

In [33]:
for row in ratings_test.itertuples():
    data_model_test_matrix.loc[row[1]][row[2]] = row[3]
data_model_test_matrix = data_model_test_matrix.fillna(0)

In [34]:
data_model_test_matrix

,289,290,291,292,293,294,295,296,297,298,...,741,742,743,744,745,746,747,748,749,750
15,0.380952,0.285714,0.615385,0.363636,0.615385,0.363636,0.615385,0.363636,0.615385,0.363636,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
39,0.421053,0.200000,0.666667,0.222222,0.666667,0.222222,0.666667,0.222222,0.666667,0.222222,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,0.434783,0.363636,0.833333,0.444444,0.833333,0.444444,0.833333,0.444444,0.833333,0.444444,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
22,0.200000,0.210526,0.235294,0.235294,0.235294,0.235294,0.235294,0.235294,0.235294,0.235294,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
128,0.285714,0.285714,0.285714,0.285714,0.285714,0.285714,0.285714,0.285714,0.285714,0.285714,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
103,0.666667,0.333333,0.666667,0.333333,0.666667,0.333333,0.666667,0.333333,0.666667,0.333333,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
116,0.800000,0.285714,0.750000,0.333333,0.750000,0.333333,0.750000,0.333333,0.750000,0.333333,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
113,0.444444,0.250000,0.571429,0.333333,0.571429,0.333333,0.571429,0.333333,0.571429,0.333333,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
111,0.285714,0.333333,0.333333,0.333333,0.333333,0.333333,0.333333,0.333333,0.333333,0.333333,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
138,0.608696,0.333333,0.823529,0.333333,0.750000,0.333333,0.750000,0.333333,0.750000,0.333333,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


Dataset Similarity Matrix

In [35]:
meta_datasets = pd.read_csv("./Data/dataset_v.csv",low_memory=False)
datasets = meta_datasets.dataset_id.unique()
meta_datasets = meta_datasets.loc[:,("v1","v2","v3","v4","v5")]

In [36]:
meta_dataset_similarity = cosine_similarity(meta_datasets.values.tolist())
meta_dataset_similarity = pd.DataFrame(meta_dataset_similarity)

In [37]:
meta_models

,model_id,model_type,v1,v2,v3,v4,distance,threshold,prominence,height,width
0,289,0,0.124729,0.109505,-0.132957,0.163413,1,0.3,0,0,0
1,290,0,0.124729,0.109505,-0.132957,0.163413,1,0.7,0,0,0
2,291,0,0.124729,0.109505,-0.132957,0.163413,100,0.3,0,0,0
3,292,0,0.124729,0.109505,-0.132957,0.163413,100,0.7,0,0,0
4,293,0,0.124729,0.109505,-0.132957,0.163413,150,0.3,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
457,746,1,0.123656,0.107174,-0.123199,0.164318,350,0.5,400,50,5
458,747,1,0.123656,0.107174,-0.123199,0.164318,350,0.0,1000,50,0
459,748,1,0.123656,0.107174,-0.123199,0.164318,350,0.0,1000,50,5
460,749,1,0.123656,0.107174,-0.123199,0.164318,350,0.5,1000,50,0


In [38]:
meta_models = meta_models.loc[:,("v1","v2","v3","v4")]

In [39]:
meta_model_similarity = cosine_similarity(meta_models.values.tolist())
meta_model_similarity = pd.DataFrame(meta_model_similarity,index=models,columns=models)

In [40]:
# user_similarity = cosine_similarity(data_model_train_matrix)
# datasets_similarity = pd.DataFrame(user_similarity,index=datasets_train,columns=datasets_train)
def cosine_similarity_func(ratings, user1, user2):
    # 找到两个用户共同评分的物品，并将这些评分放入一个向量中
    cols = ratings.columns[ratings.loc[user1].values.nonzero()[0]]
    common_items = np.intersect1d(cols, ratings.columns[ratings.loc[user2].values.nonzero()[0]]).tolist()
    u1_ratings = ratings.loc[user1,common_items]
    u2_ratings = ratings.loc[user2,common_items]
    # 计算两个向量之间的余弦相似度
    if len(common_items) == 0:
        return 0
    else:
        cos_sim = np.dot(u1_ratings, u2_ratings) / (np.linalg.norm(u1_ratings) * np.linalg.norm(u2_ratings))
        return cos_sim


dataset_similarity = pd.DataFrame(index=datasets_train, columns=datasets_train)
for i in datasets_train:
    for j in datasets_train:
        dataset_similarity.loc[i][j] = cosine_similarity_func(data_model_train_matrix,i,j)

Prediction

In [41]:
# def predict(ratings, similarity):
#         mean_user_rating = ratings.mean(axis=1)
#         ratings_diff = ratings - np.array(mean_user_rating)[:,np.newaxis]
#         pred = np.array(mean_user_rating)[:,np.newaxis] + np.dot(similarity,ratings_diff) / np.array([np.abs(similarity).sum(axis = 1)]).T
#         return pred

def predict(ratings, user_similarities, user_id, item_id):
    # 获取与用户最相似的K个用户
    k = 1
    similar_users = user_similarities.loc[user_id].sort_values(ascending=False).index.tolist()[1:k+1]
    # print(similar_users)
    # 计算加权平均评分
    weighted_sum = 0
    sum_of_weights = 0
    for sim_user_id in similar_users:
        if ratings.loc[sim_user_id][item_id] != 0:
            # print(sim_user_id,item_id)
            weighted_sum += user_similarities.loc[user_id][sim_user_id] * ratings.loc[sim_user_id][item_id]
            sum_of_weights += user_similarities.loc[user_id][sim_user_id]
    if sum_of_weights == 0:
        return 0
    else:
        return weighted_sum / sum_of_weights

In [42]:
model_prediction_train = pd.DataFrame(index=datasets_train,columns=models)

In [43]:
for i in datasets_train:
    for j in models:
        if data_model_train_matrix.loc[i][j] == 0:
            model_prediction_train.loc[i][j] = predict(data_model_train_matrix,dataset_similarity,i,j)

In [44]:
# train_prediction = predict(data_model_train_matrix,user_similarities)
model_prediction_train = pd.DataFrame(data_model_train_matrix,index=datasets_train,columns=models).sort_index().sort_index(axis=1)
model_prediction_test = pd.DataFrame(index=datasets_test,columns=model_test)

Metadata Based

In [45]:
def find_sim_index(index):
    row1 = meta_dataset_similarity.loc[index]
    row1_max_index = row1[row1 == row1.max()].index[0]
    return row1_max_index

In [46]:
def Find_Top_k(i,sim_matrix):
    row = sim_matrix.loc[i]
    row = row.sort_values(ascending=False)
    index_row = row.index
    index_row = index_row.values.tolist()
    return index_row

In [47]:
start_time = time.time()

In [48]:
for dataset in datasets_test:
    for model in model_test:
        dataset_sim_list = Find_Top_k(dataset,meta_dataset_similarity)[1:]
        for sim_dataset in dataset_sim_list:
            if sim_dataset not in datasets_train:
                continue
            if model_prediction_train.loc[sim_dataset][model] > 0.1:
                model_prediction_test.loc[dataset][model] = model_prediction_train.loc[sim_dataset][model]
                break

In [49]:
end_time = time.time()

In [50]:
end_time - start_time

24.83176565170288

In [51]:
for i in datasets_test:
    for j in model_test:
        if data_model_test_matrix.loc[i][j] == 0:
            model_prediction_test.loc[i][j] = None

In [52]:
new_result = pd.DataFrame(columns={"dataset","model","predict_F1_Score","groundtruth_F1_Score"})

In [53]:
for i in datasets_test:
    for j in model_test:
        if model_prediction_test.loc[i][j] is not None:
            predict_F1_Score = model_prediction_test.loc[i][j]
            groundtruth_F1_Score = data_model_test_matrix.loc[i][j]
            new_result = new_result.append([{'dataset':i,'model':j,'predict_F1_Score':predict_F1_Score,'groundtruth_F1_Score':groundtruth_F1_Score}],ignore_index=True)

In [54]:
new_result.fillna(0)

,model,dataset,groundtruth_F1_Score,predict_F1_Score
0,289,15,0.380952,0.500000
1,290,15,0.285714,0.181818
2,291,15,0.615385,0.714286
3,292,15,0.363636,0.200000
4,293,15,0.615385,0.714286
...,...,...,...,...
12539,746,284,0.023451,0.210526
12540,747,284,0.023451,0.216216
12541,748,284,0.023451,0.216216
12542,749,284,0.023451,0.216216


In [55]:
new_result.to_csv("../Peak_Finding/Output/full.csv",index=False)